In [1]:
import numpy as np
from sklearn.model_selection import train_test_split

from cs330_project.datasets import RawDataset

rng = np.random.default_rng()
img_size = 32
n_classes = 10
n, c, t, h, w = 20, 3, 10, img_size, img_size
x_data = rng.random((n, c, t, h, w), dtype=np.float32)
y_data = rng.integers(0, n_classes, n, dtype=np.int32)
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.20)
dataset_train = RawDataset(x_train, y_train)
dataset_test = RawDataset(x_test, y_test)

In [2]:
from cs330_project.models import ViTAutoEncoder

model = ViTAutoEncoder(
    in_img_size=32,
    in_channels=3,
    patch_size=4,
    spatio_temporal=True,
    tubelet_size=2,
    in_num_frames=10,
    encoder_embed_dim=192,
    encoder_depth=9,
    encoder_num_heads=12,
    decoder_embed_dim=96,
    decoder_depth=3,
    decoder_num_heads=16,
    mlp_dim_ratio=2,
    head_dim=192 // 12,
    is_spt=False,
    is_lsa=False)

In [3]:
from cs330_project.training import train_one_epoch, make_optimizer

optimizer = make_optimizer(model)


In [4]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
from cs330_project.datasets import DataLoader
dataloader_train = DataLoader(
    dataset_train,
    batch_size=2,
    num_workers=0,
    pin_memory=False,
    drop_last=True)


In [6]:
train_one_epoch(model, None, dataloader_train, optimizer, device)

ValueError: not enough values to unpack (expected 3, got 2)